In [1]:
from thirdai import bolt
from thirdai.demos import download_beir_dataset

(
    unsupervised_file,
    supervised_trn,
    supervised_tst,
    n_target_classes,
) = download_beir_dataset("scifact")

model = bolt.UniversalDeepTransformer(
    data_types={
        "QUERY": bolt.types.text(contextual_encoding="local"),
        "DOC_ID": bolt.types.categorical(delimiter=":"),
    },
    target="DOC_ID",
    n_target_classes=n_target_classes,
    integer_target=True,
    options={"extreme_classification": True, "embedding_dimension": 512},
)

metrics = model.cold_start(
    filename=unsupervised_file,
    strong_column_names=["TITLE"],
    weak_column_names=["TEXT"],
    learning_rate=0.001,
    epochs=5,
    metrics=[
        "precision@1",
        "recall@10",
    ],
)

validation = bolt.Validation(
    supervised_tst,
    metrics=["precision@1"],
)

metrics = model.train(
    filename=supervised_trn,
    learning_rate=0.001,
    epochs=10,
    metrics=[
        "precision@1",
        "recall@10",
    ],
    validation=validation,
)

model.save("mach_scifact.bolt")

ModuleNotFoundError: No module named 'thirdai'

In [ ]:
from thirdai import bolt
model = bolt.UniversalDeepTransformer.load("mach_scifact.bolt")

model.predict({"QUERY": "microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging"})
model.predict({"QUERY": "beyonce jayz and other artists have gathered to perform music at the concert."})
model.introduce([{"QUERY": "beyonce jayz and other artists have gathered to perform music at the concert."}], 5183)
model.predict({"QUERY": "beyonce jayz and other artists have gathered to perform music at the concert."})
model.predict({"QUERY": "are beyonce and jayz performing music at the concert ?"})
model.forget(5183)


model.introduce([{"QUERY": "beyonce is an artist that performs music."}], 5184)